<a href="https://colab.research.google.com/github/ashev2021/Fine-Tuning-LLM/blob/main/SetFit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install "datasets>=2.18.0,<3"
# deploy in Gradio
!pip install gradio

In [ ]:
!pip install transformers>=4.38.2 sentence-transformers>=2.5.1 setfit>=1.0.3 accelerate>=0.27.2 seqeval>=1.2.2

In [ ]:
#Load dataset
from datasets import load_dataset

from setfit import sample_dataset

# Prepare data and splits
imdb_data = load_dataset("mteb/imdb")

train_data, test_data = imdb_data["train"], imdb_data["test"]


train_data = train_data.shuffle(seed=42).select(range(2000))
test_data = test_data.shuffle(seed=42).select(range(1000))


# We simulate a few-shot setting by sampling 16 examples per class
sampled_train_data = sample_dataset(imdb_data["train"], num_samples=16)

In [ ]:
from setfit import SetFitModel

# Load a pre-trained SentenceTransformer model
model = SetFitModel.from_pretrained("sentence-transformers/distilbert-base-nli-mean-tokens")

In [ ]:
# evaluate the model with F1
import numpy as np
import evaluate


def compute_metrics(eval_pred):
    """Calculate F1 score"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    load_f1 = evaluate.load("f1")
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"f1": f1}


In [ ]:
from setfit import TrainingArguments as SetFitTrainingArguments
from setfit import Trainer as SetFitTrainer

# Define training arguments
args = SetFitTrainingArguments(
    num_epochs=2, # The number of epochs to use for contrastive learning
    num_iterations=20  # The number of text pairs to generate
)
args.eval_strategy = args.evaluation_strategy

# Create trainer
trainer = SetFitTrainer(
    model=model,
    args=args,
    train_dataset=sampled_train_data,
    eval_dataset=test_data,
    metric="f1"
)

In [ ]:
import wandb

wandb.init(project="your-project", name="run-1")



trainer.train()


In [ ]:
trainer.evaluate()

In [ ]:
#save fine-tuned model
trainer.model.save_pretrained("SetFit_finetuned_model")


In [ ]:
import gradio as gr
from setfit import SetFitModel
from transformers import pipeline

# Load your fine-tuned SetFit model
fine_tuned_model = SetFitModel.from_pretrained("./SetFit_finetuned_model")  # Use "./" for local

# Load a pretrained sentiment model
pretrained_classifier = pipeline("sentiment-analysis", model="sentence-transformers/distilbert-base-nli-mean-tokens")

# Label map for display purposes (adjust to your model’s labels if different)
label_map = {
    "LABEL_0": "negative",
    "LABEL_1": "positive",
    "NEGATIVE": "negative",
    "POSITIVE": "positive",
    0: "negative",
    1: "positive"
}


def compare_models(text):
    # Fine-tuned SetFit model
    ft_pred = fine_tuned_model.predict([text])[0]
    ft_pred = int(ft_pred)  # Convert tensor to int

    ft_label = label_map.get(ft_pred, str(ft_pred))

    ft_proba = fine_tuned_model.predict_proba([text])[0]
    ft_confidence = round(float(max(ft_proba)), 4)  # Convert tensor to float

    ft_result = f"Label: {ft_label}, Confidence: {ft_confidence}"

    # Pretrained pipeline model
    pre_pred = pretrained_classifier(text)[0]
    pre_label = label_map.get(pre_pred["label"], pre_pred["label"])
    pre_score = round(pre_pred["score"], 4)
    pre_result = f"Label: {pre_label}, Confidence: {pre_score}"

    return ft_result, pre_result

with gr.Blocks() as demo:
    gr.Markdown("## 🎬 Compare Fine-Tuned(SetFit) vs Pretrained Models")
    gr.Markdown("Enter a movie review below to compare predictions.")

    input_text = gr.Textbox(label="Movie Review", lines=4, placeholder="Type your review here...")
    classify_btn = gr.Button("Classify")

    with gr.Row():
        ft_output = gr.Textbox(label="Fine-Tuned Model Prediction", interactive=False)
        pre_output = gr.Textbox(label="Pretrained Model Prediction", interactive=False)

    classify_btn.click(compare_models, inputs=input_text, outputs=[ft_output, pre_output])

demo.launch(share=True)
